In [1]:
%pip install pytorch-tabnet

import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold

import torch
from pytorch_tabnet.pretraining import TabNetPretrainer
from pytorch_tabnet.tab_model import TabNetRegressor

   ---------------------------------------- 0.0/212.5 MB ? eta -:--:--
   ---------------------------------------- 2.1/212.5 MB 11.8 MB/s eta 0:00:18
    --------------------------------------- 4.7/212.5 MB 11.4 MB/s eta 0:00:19
   - -------------------------------------- 6.8/212.5 MB 11.0 MB/s eta 0:00:19
   - -------------------------------------- 9.2/212.5 MB 11.2 MB/s eta 0:00:19
   -- ------------------------------------- 11.5/212.5 MB 11.3 MB/s eta 0:00:18
   -- ------------------------------------- 13.9/212.5 MB 11.0 MB/s eta 0:00:18
   --- ------------------------------------ 16.3/212.5 MB 11.1 MB/s eta 0:00:18
   --- ------------------------------------ 18.6/212.5 MB 11.2 MB/s eta 0:00:18
   --- ------------------------------------ 21.0/212.5 MB 11.2 MB/s eta 0:00:18
   ---- ----------------------------------- 23.3/212.5 MB 11.2 MB/s eta 0:00:17
   ---- ----------------------------------- 25.7/212.5 MB 11.2 MB/s eta 0:00:17
   ----- ---------------------------------- 28.0/212.

In [85]:
!pip install --upgrade xgboost

   ---------------------------------------- 0.0/150.0 MB ? eta -:--:--
    --------------------------------------- 2.4/150.0 MB 11.2 MB/s eta 0:00:14
   - -------------------------------------- 4.7/150.0 MB 11.4 MB/s eta 0:00:13
   - -------------------------------------- 7.1/150.0 MB 11.5 MB/s eta 0:00:13
   -- ------------------------------------- 9.4/150.0 MB 11.5 MB/s eta 0:00:13
   --- ------------------------------------ 12.1/150.0 MB 11.6 MB/s eta 0:00:12
   --- ------------------------------------ 14.4/150.0 MB 11.6 MB/s eta 0:00:12
   ---- ----------------------------------- 17.0/150.0 MB 11.7 MB/s eta 0:00:12
   ----- ---------------------------------- 19.4/150.0 MB 11.7 MB/s eta 0:00:12
   ----- ---------------------------------- 22.0/150.0 MB 11.6 MB/s eta 0:00:12
   ------ --------------------------------- 24.4/150.0 MB 11.7 MB/s eta 0:00:11
   ------- -------------------------------- 26.7/150.0 MB 11.6 MB/s eta 0:00:11
   ------- -------------------------------- 29.1/150.

In [161]:
df_sample =  pd.read_csv("data_files/sample_submission.csv")
df_test =  pd.read_csv("data_files/test.csv")
df_train =  pd.read_csv("data_files/train.csv")

In [78]:
len(df_sample)
# df_test.info()
# df_train.info()

1755

In [162]:
df_train

,ID,설립연도,국가,분야,투자단계,직원 수,인수여부,상장여부,고객수(백만명),총 투자금(억원),연매출(억원),SNS 팔로워 수(백만명),기업가치(백억원),성공확률
0,TRAIN_0000,2009,CT005,이커머스,Series A,4126.0,No,No,56.0,3365.0,4764.0,4.71,NaN,0.3
1,TRAIN_0001,2023,CT006,핀테크,Seed,4167.0,Yes,No,80.0,4069.0,279.0,1.00,2500-3500,0.8
2,TRAIN_0002,2018,CT007,기술,Series A,3132.0,Yes,Yes,54.0,6453.0,12141.0,4.00,3500-4500,0.5
3,TRAIN_0003,2016,CT006,NaN,Seed,3245.0,Yes,Yes,NaN,665.0,10547.0,2.97,NaN,0.7
4,TRAIN_0004,2020,CT002,에듀테크,Seed,1969.0,No,Yes,94.0,829.0,9810.0,1.00,1500-2500,0.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4371,TRAIN_4371,2021,CT006,푸드테크,Series A,4841.0,Yes,No,90.0,4187.0,9394.0,4.00,1500-2500,0.8
4372,TRAIN_4372,2020,CT003,에너지,Series B,555.0,No,Yes,37.0,796.0,2969.0,3.00,6000이상,0.4
4373,TRAIN_4373,2023,CT004,NaN,Series C,506.0,No,Yes,NaN,3314.0,4512.0,1.47,NaN,0.6
4374,TRAIN_4374,2001,CT007,AI,IPO,1438.0,No,No,53.0,2395.0,3755.0,5.00,4500-6000,0.9


In [163]:
df_train.isna().sum()
df_train['투자단계'].unique()
# df_train['국가'].unique()

array(['Series A', 'Seed', 'Series C', 'Series B', 'IPO'], dtype=object)

In [164]:
df_train["유지기간"] = (2025 - df_train["설립연도"])

In [165]:
cols = ["설립연도","국가","분야","직원 수","인수여부","상장여부","고객수(백만명)","총 투자금(억원)","연매출(억원)","SNS 팔로워 수(백만명)","기업가치(백억원)","유지기간",'투자단계']
X_train = df_train[cols]
y_train = df_train['성공확률']

X_train

,설립연도,국가,분야,직원 수,인수여부,상장여부,고객수(백만명),총 투자금(억원),연매출(억원),SNS 팔로워 수(백만명),기업가치(백억원),유지기간,투자단계
0,2009,CT005,이커머스,4126.0,No,No,56.0,3365.0,4764.0,4.71,NaN,16,Series A
1,2023,CT006,핀테크,4167.0,Yes,No,80.0,4069.0,279.0,1.00,2500-3500,2,Seed
2,2018,CT007,기술,3132.0,Yes,Yes,54.0,6453.0,12141.0,4.00,3500-4500,7,Series A
3,2016,CT006,NaN,3245.0,Yes,Yes,NaN,665.0,10547.0,2.97,NaN,9,Seed
4,2020,CT002,에듀테크,1969.0,No,Yes,94.0,829.0,9810.0,1.00,1500-2500,5,Seed
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4371,2021,CT006,푸드테크,4841.0,Yes,No,90.0,4187.0,9394.0,4.00,1500-2500,4,Series A
4372,2020,CT003,에너지,555.0,No,Yes,37.0,796.0,2969.0,3.00,6000이상,5,Series B
4373,2023,CT004,NaN,506.0,No,Yes,NaN,3314.0,4512.0,1.47,NaN,2,Series C
4374,2001,CT007,AI,1438.0,No,No,53.0,2395.0,3755.0,5.00,4500-6000,24,IPO


In [166]:
# X_train.isna().sum()
X_train['투자단계'].unique()
most_value = df_train['기업가치(백억원)'].mode()[0]
# most_value


In [167]:
X_train.loc[:, '국가'] = X_train['국가'].map({'CT005' : 5, 'CT006': 6, 'CT007': 7, 'CT002': 2, 'CT008': 8, 'CT010': 10, 'CT001': 1,
       'CT009': 9, 'CT003': 3, 'CT004' : 4})
X_train.loc[:, '분야'] = X_train['분야'].fillna("분야없음")
X_train.loc[:, '분야'] = X_train['분야'].map({'이커머스':1, '핀테크':2, '기술':3, '분야없음':4, '에듀테크':5, '게임':6, '헬스케어':7, '물류':8, '푸드테크':9, 'AI':10,
       '에너지':11})
X_train.loc[:, '직원 수'] = X_train['직원 수'].fillna(X_train['직원 수'].mean())
X_train.loc[:, '인수여부'] = X_train['인수여부'].map({'Yes':1,'No':2})
X_train.loc[:, '상장여부'] = X_train['상장여부'].map({'Yes':1,'No':2})
X_train.loc[:, '고객수(백만명)'] = X_train['고객수(백만명)'].fillna(X_train['고객수(백만명)'].mean())
X_train['기업가치(백억원)'] = X_train['기업가치(백억원)'].fillna(most_value)
X_train.loc[:, '기업가치(백억원)'] = X_train['기업가치(백억원)'].map({'2500-3500':1, '3500-4500':2, '1500-2500':3, '4500-6000':4, '6000이상':5})
X_train.loc[:, '투자단계'] = X_train['투자단계'].map({'Series A':1, 'Seed':2, 'Series C':3, 'Series B':4, 'IPO':5})


C:\Users\human\AppData\Local\Temp\ipykernel_6812\2965940614.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['기업가치(백억원)'] = X_train['기업가치(백억원)'].fillna(most_value)


In [133]:
X_train

,설립연도,국가,분야,직원 수,인수여부,상장여부,고객수(백만명),총 투자금(억원),연매출(억원),SNS 팔로워 수(백만명),기업가치(백억원),유지기간,투자단계
0,2009,5,1,4126.0,2,2,56.000000,3365.0,4764.0,4.71,4,16,1
1,2023,6,2,4167.0,1,2,80.000000,4069.0,279.0,1.00,1,2,2
2,2018,7,3,3132.0,1,1,54.000000,6453.0,12141.0,4.00,2,7,1
3,2016,6,4,3245.0,1,1,49.214332,665.0,10547.0,2.97,4,9,2
4,2020,2,5,1969.0,2,1,94.000000,829.0,9810.0,1.00,3,5,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4371,2021,6,9,4841.0,1,2,90.000000,4187.0,9394.0,4.00,3,4,1
4372,2020,3,11,555.0,2,1,37.000000,796.0,2969.0,3.00,5,5,4
4373,2023,4,4,506.0,2,1,49.214332,3314.0,4512.0,1.47,4,2,3
4374,2001,7,10,1438.0,2,2,53.000000,2395.0,3755.0,5.00,4,24,5


In [ ]:
# X_train.info()
# X_train[['국가','분야','인수여부','상장여부','기업가치(백억원)','투자단계']] = X_train[['국가','분야','인수여부','상장여부','기업가치(백억원)','투자단계']].astype(int)
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4376 entries, 0 to 4375
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   설립연도            4376 non-null   int64  
 1   국가              4376 non-null   int64  
 2   분야              4376 non-null   int64  
 3   직원 수            4376 non-null   float64
 4   인수여부            4376 non-null   int64  
 5   상장여부            4376 non-null   int64  
 6   고객수(백만명)        4376 non-null   float64
 7   총 투자금(억원)       4376 non-null   float64
 8   연매출(억원)         4376 non-null   float64
 9   SNS 팔로워 수(백만명)  4376 non-null   float64
 10  기업가치(백억원)       4376 non-null   int64  
 11  유지기간            4376 non-null   int64  
 12  투자단계            4376 non-null   int64  
dtypes: float64(5), int64(8)
memory usage: 444.6 KB


C:\Users\human\AppData\Local\Temp\ipykernel_6812\1507380351.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[['국가','분야','인수여부','상장여부','기업가치(백억원)','투자단계']] = X_train[['국가','분야','인수여부','상장여부','기업가치(백억원)','투자단계']].astype(int)


In [168]:
from sklearn.linear_model import LinearRegression
# from xgboost import XGBRegressor

# model = XGBRegressor()
# model.fit(X_train, y_train)

In [169]:
model = LinearRegression()
model.fit(X_train,y_train)

LinearRegression()

In [170]:
df_test["유지기간"] = (2025 - df_test["설립연도"])

In [171]:
X_test = df_test[cols]


In [172]:
X_test.isna().sum()
# X_test
# df_train['상장여부'].unique()
most_value2 = df_test['기업가치(백억원)'].mode()[0]
# most_value2

In [173]:
X_test.loc[:, '국가'] = X_test['국가'].map({'CT005' : 5, 'CT006': 6, 'CT007': 7, 'CT002': 2, 'CT008': 8, 'CT010': 10, 'CT001': 1,
       'CT009': 9, 'CT003': 3, 'CT004' : 4})
X_test.loc[:, '분야'] = X_test['분야'].fillna("분야없음")
X_test.loc[:, '분야'] = X_test['분야'].map({'이커머스':1, '핀테크':2, '기술':3, '분야없음':4, '에듀테크':5, '게임':6, '헬스케어':7, '물류':8, '푸드테크':9, 'AI':10,
       '에너지':11})
X_test.loc[:, '직원 수'] = X_test['직원 수'].fillna(X_test['직원 수'].mean())
X_test.loc[:, '인수여부'] = X_test['인수여부'].map({'Yes':1,'No':2})
X_test.loc[:, '상장여부'] = X_test['상장여부'].map({'Yes':1,'No':2})
X_test.loc[:, '고객수(백만명)'] = X_test['고객수(백만명)'].fillna(X_test['고객수(백만명)'].mean())
X_test['기업가치(백억원)'] = X_test['기업가치(백억원)'].fillna(most_value2)
X_test.loc[:, '기업가치(백억원)'] = X_test['기업가치(백억원)'].map({'2500-3500':1, '3500-4500':2, '1500-2500':3, '4500-6000':4, '6000이상':5})
X_test.loc[:, '투자단계'] = X_test['투자단계'].map({'Series A':1, 'Seed':2, 'Series C':3, 'Series B':4, 'IPO':5})


C:\Users\human\AppData\Local\Temp\ipykernel_6812\206279615.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['기업가치(백억원)'] = X_test['기업가치(백억원)'].fillna(most_value2)


In [ ]:
# X_test[['국가','분야','인수여부','상장여부','기업가치(백억원)','투자단계']] = X_test[['국가','분야','인수여부','상장여부','기업가치(백억원)','투자단계']].astype(int)

C:\Users\human\AppData\Local\Temp\ipykernel_6812\168076305.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test[['국가','분야','인수여부','상장여부','기업가치(백억원)','투자단계']] = X_test[['국가','분야','인수여부','상장여부','기업가치(백억원)','투자단계']].astype(int)


In [174]:
prediction = model.predict(X_test)

In [175]:
prediction
df_sample['성공확률'] = prediction
df_sample

,ID,성공확률
0,TEST_0000,0.554770
1,TEST_0001,0.523347
2,TEST_0002,0.541890
3,TEST_0003,0.540853
4,TEST_0004,0.544390
...,...,...
1750,TEST_1750,0.551570
1751,TEST_1751,0.544112
1752,TEST_1752,0.529996
1753,TEST_1753,0.559967


In [176]:
df_sample.to_csv("data_files/submission4.csv", index=False)